### Naive Bayes Classifier - twitter data

#### Bag of Words Approach

In [1]:
import twitter
import json

#CONSUMER_KEY = ''
#CONSUMER_SECRET = ''

#OAUTH_TOKEN = ''
#OAUTH_TOKEN_SECRET = ''

CONSUMER_KEY = 'KzLNf2VNYhdmbEeCl0E5lGAnh'
CONSUMER_SECRET = 'PRRteY7pNeAdfC73MR469IH10JkgMwIRkelvvR4eVpiYfAo3Bh'

OAUTH_TOKEN = '3183721-F6qIQaeelhdfW2ZTFRwXwJDmvSHWgvQcatqMK3DXee'
OAUTH_TOKEN_SECRET = 'Itwk1uGJM1hWQGpWSAGCchIZ5hwOZ3UsX4iGsFvTUhOcQ'

auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                           CONSUMER_KEY, CONSUMER_SECRET)

twitter_api = twitter.Twitter(auth=auth)

In [2]:
q1 = 'donald trump' 
count = 100

# See https://dev.twitter.com/docs/api/1.1/get/search/tweets
search_results = twitter_api.search.tweets(q=q1, count=count)
q1_texts = search_results['statuses']

# use a loop
num_iterations = 10

for i in range(num_iterations):
    params = {a:b for a,b in [x.split('=') for x in search_results['search_metadata']['next_results'][1:].split('&')]}
    max_id = int(params['max_id'])    
    search_results = twitter_api.search.tweets(q=q1, count=count, max_id=max_id)
    q1_texts += search_results['statuses']

In [3]:
q1_texts = [status['text'] for status in q1_texts]
words_trump = [w.lower() for t in q1_texts for w in t.split()]

In [4]:
q1_texts[:10]

[u'RT @billybragg: Time to stop taking the candidacy of Donald Trump as a joke https://t.co/0B4SD0hn3o',
 u"I'm voting for Donald Trump because I never seen a president get assassinated before",
 u'RT @iLikeSexDaily: a mosaic of donald trump created from over 5,000 dick pics https://t.co/o28oQ7DnOC',
 u'RT @CraigRozniecki: "Donald Trump\'s Plan for a Muslim Database Draws Comparisons to Nazi Germany" by Vaughn Hillyard - https://t.co/c5ljVfY\u2026',
 u'RT @iLikeSexDaily: a mosaic of donald trump created from over 5,000 dick pics https://t.co/o28oQ7DnOC',
 u'RT @merimartusa: adolf trump / donald hitler https://t.co/22cnnQ4yJO',
 u"RT @TODAYshow: Donald Trump's plan for a Muslim database draws comparison to Nazi Germany: https://t.co/MzgLov1sgV https://t.co/NR3YSeuYcG",
 u"RT @counternotions: Donald Trump may or may not be a fascist, but he isn't quite the problem. \n\nTens of millions of Americans who may vote \u2026",
 u'RT @HasnaatMahmood: Donald Trump is literally hitler reincarnated

In [5]:
q2 = 'kardashian' 
count = 100

# See https://dev.twitter.com/docs/api/1.1/get/search/tweets
search_results = twitter_api.search.tweets(q=q2, count=count)
q2_tweets = search_results['statuses']

# use a loop
num_iterations = 10

for i in range(num_iterations):
    params = {a:b for a,b in [x.split('=') for x in search_results['search_metadata']['next_results'][1:].split('&')]}
    max_id = int(params['max_id'])    
    search_results = twitter_api.search.tweets(q=q2, count=count, max_id=max_id)
    q2_tweets += search_results['statuses']

In [6]:
q2_texts = [status['text'] for status in q2_tweets]
words_kardashian = [w.lower() for t in q2_texts for w in t.split()]

In [7]:
q2_texts[:10]

[u'Photos of Kim Kardashian Before Was Famous\U0001f602\U0001f633 #FridayFeeling https://t.co/1yuTpH4pzK',
 u"I'm now a D-List++ celebrity in Kim Kardashian: Hollywood. You can be famous too by playing on Android! https://t.co/eFJyyj7Eii",
 u'Kerry Washington, Katie Holmes, Kourtney Kardashian Party at the Stylemakers Awards https://t.co/fCRhZk68ir',
 u'Forget Kim Kardashian! Meet Anastasia Kvitko',
 u'@asherahresearch If this were true, Justin Bieber or - God beware - Kim Kardashian would be omniscient. Yuk!',
 u"I'm now an A-List celebrity in Kim Kardashian: Hollywood. You can be famous too by playing on Android! https://t.co/brZsqXJqcX",
 u"I'm now a B-List celebrity in Kim Kardashian: Hollywood. You can be famous too by playing on iPhone!  https://t.co/Mud5zO2cKZ",
 u'Moniek 3 Erwin. @erwin_moniek Appealing Kim Kardashian West Images. Get started now: https://t.co/oOWGXIChEj',
 u'RT @shainamnbg_: dude, ur a desperate piece of shit',
 u'RT @brttmrtn: #NoteToSelf ; wag ipagpilitan an

### prepare our featuresets

In [8]:
import nltk
from nltk.corpus import stopwords
import re

stub = re.compile('[^A-Za-z]')
common_words = ['rt']

def bag_of_non_stopwords(text):
    # create array of words
    words = [stub.sub('', w).lower() for w in text.split()]

    # remove English stop words and common words
    finalwords = set(words) - set(stopwords.words('english')) - set(common_words)
    
    # format feature set data object
    featureset = dict([(word, True) for word in finalwords if not word.startswith('http') and len(word)>2])
    
    return featureset

bag_of_non_stopwords(q1_texts[30])

{u'donald': True,
 u'favor': True,
 u'hes': True,
 u'jpm': True,
 u'muslims': True,
 u'registrationdatabase': True,
 u'say': True,
 u'trump': True,
 u'via': True}

In [9]:
from collections import defaultdict

def extract_features(text):
    return bag_of_non_stopwords(text)

label_features = defaultdict(list)

for tweet_text in q2_texts:
    features = extract_features(tweet_text)
    label_features['kardashian'].append(features)

In [10]:
label_features['kardashian'][1]

{u'android': True,
 u'celebrity': True,
 u'dlist': True,
 u'famous': True,
 u'hollywood': True,
 u'kardashian': True,
 u'kim': True,
 u'playing': True}

In [11]:
for tweet_text in q1_texts:
    features = extract_features(tweet_text)
    label_features['trump'].append(features)

In [12]:
label_features['trump'][3]

{u'comparisons': True,
 u'craigrozniecki': True,
 u'database': True,
 u'donald': True,
 u'draws': True,
 u'germany': True,
 u'hillyard': True,
 u'muslim': True,
 u'nazi': True,
 u'plan': True,
 u'trumps': True,
 u'vaughn': True}

In [13]:
label_features.keys()

['kardashian', 'trump']

In [14]:
# split the data into training / test sets -> 75/25% split
def split_label_feats(lfeats, split=0.75):
    train_feats = []
    test_feats = []

    for label, feats in lfeats.iteritems():
        cutoff = int(len(feats) * split)
        train_feats.extend([(feat, label) for feat in feats[:cutoff]])
        test_feats.extend([(feat, label) for feat in feats[cutoff:]])

    return train_feats, test_feats

In [15]:
# Split data into training/testing - (split = 0.75) by default
train_feats, test_feats = split_label_feats(label_features)

print len(train_feats)
print len(test_feats)

1650
550


In [16]:
# Naive Bayes Classifier
from nltk.classify import NaiveBayesClassifier

nb = NaiveBayesClassifier.train(train_feats)

In [17]:
ex1_features = extract_features("protect us from terrorists")
nb.classify(ex1_features)

'trump'

In [18]:
ex2_features = extract_features("pregnant with another baby !@#!@")
nb.classify(ex2_features)

'kardashian'

In [19]:
# Calculate accuracy
from nltk.classify.util import accuracy

accuracy(nb, test_feats)

0.9945454545454545

In [20]:
# getting classification probability of each label
probs = nb.prob_classify(test_feats[6][0])

print probs.samples()
print 'chosen class:',probs.max()
print 'P(trump)=',probs.prob('trump')
print 'P(kardashian)=',probs.prob('kardashian')

['kardashian', 'trump']
chosen class: kardashian
P(trump)= 2.24713921377e-11
P(kardashian)= 0.999999999978


In [21]:
test_feats[6]

({u'goes': True,
  u'kardashian': True,
  u'khloe': True,
  u'lamar': True,
  u'man': True,
  u'odom': True,
  u'rant': True,
  u'twitter': True,
  u'vows': True,
  u'went': True},
 'kardashian')

In [22]:
# most informative feature - feature value will always be True

nb.most_informative_features(10)

[(u'kardashian', True),
 (u'donald', True),
 (u'kim', True),
 (u'trump', True),
 (u'asshole', True),
 (u'president', True),
 (u'kids', True),
 (u'would', True),
 (u'latest', True),
 (u'away', True)]

In [23]:
# more information on the chosen label

nb.show_most_informative_features(10)

Most Informative Features
              kardashian = True           kardas : trump  =    453.7 : 1.0
                  donald = True            trump : kardas =    188.1 : 1.0
                     kim = True           kardas : trump  =    183.0 : 1.0
                   trump = True            trump : kardas =    175.3 : 1.0
                 asshole = True           kardas : trump  =     48.3 : 1.0
               president = True            trump : kardas =     32.3 : 1.0
                    kids = True           kardas : trump  =     23.7 : 1.0
                   would = True            trump : kardas =     20.6 : 1.0
                  latest = True           kardas : trump  =     19.8 : 1.0
                    away = True           kardas : trump  =     19.4 : 1.0
